# **Call 통계**

- 커서의 활동상태를 Parse, Execute, Fetch 세 단계로 나누어 각각에 대한 수행통계를 보여줌.

**Call 종류**

- **Parse Call**
  - 커서를 파싱하는 과정에 대한 통계.
  - 실행계획을 생성하거나 찾는 과정에 관한 정보를 포함.
- **Execute Call**
  - 커서를 실행하는 단계에 대한 통계.
- **Fetch Call**
  - select문에서 실제 레코드를 읽어 사용자가 요구한 결과집합을 반환하는 과정에 대한 통계.
  - insert, update, delete, merge 등 DML문은 Execute Call 시점에 모든 처리과정을 서버내에서 완료하고 처리결과만 리턴하므로 Fetch Call이 전혀 발생하지 않음.

**DML에 따른 Call 통계 예시**

**ex.1 delete**
- Parse Call : 1
- Execute Call : 1

```sql
call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        1      0.00       0.00          0         61          0           0
Execute      1      0.00       0.00          0          4          3           1
Fetch        0      0.00       0.00          0          0          0           0
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total        2      0.00       0.00          0         65          3           1
```

**ex.2 insert... select 문**

- 클라이언트로부터 명시적인 Fetch Call을 받지 않으며 서버 내에서 묵시적으로 Fetch

<br>

- Parse Call : 1
- Execute Call : 1

```sql
create table big_table as
select * from all_tables where rownum <= 10000 ;

call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        1      0.18       0.22          0          0          0           0
Execute      1      0.43       0.63         24       7659        401        2759
Fetch        0      0.00       0.00          0          0          0           0
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total        2      0.61       0.86         24       7659        401        2759
```

**ex.3 Select문**

- Select문 실행시 Excetue Call 단계에서는 커서만 오픈.
- Select문 실행시 실제 데이터를 처리하는 과정은 모두 Fetch 단계.
- sort group by는 Execute 단계에서 처리할 것으로 예상되지만 실제로는 Fetch 시점에 처리.
- 실제 데이터를 액세스하면서 일을 시작하는 시점은 첫 번째 Fetch Call 인 것을 짐작.

<br>

- Parse Call : 1
- Execute Call : 1
- Fetch Call : 1

```sql
select owner, count(*) from big_table group by owner ;

call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        1      0.00       0.00          0          1          0           0
Execute      1      0.00       0.00          0          0          0           0
Fetch        1      0.01       0.00          0         88          0          18
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total        3      0.01       0.00          0         89          0          18
```


**ex.4 for update 구문**

- Execute Call 단계에서 모든 레코드를 읽어 Lock을 설정.
- 10,000건의 레코드가 있는 테이블을 for update 구문을 사용하여 실행시켰을때 Execute 단계에서 이미 Current모드로 읽어 10,000개 레코드 전체에 대해 Lock을 설정.

<br>
- Parse Call : 1
- Execute Call : 1
- Fetch Call : 11

```sql
select * from big_table for update ;

call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        1      0.02       0.00          0          1          0           0
Execute      1      0.03       0.03          4         88      10178           0
Fetch       11      0.03       0.02          0        115          0         101
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total       13      0.08       0.06          4        204      10178         101
```